<img src="images/header-transparent.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Máster</font></h2>

<h1><font color="#6B001F" size=5>SERENDIPITY: Servicio web para la recomendacIón de playlists a partir de otra playlist</font></h1>
<h2><font color="#92002A" size=3>Parte 3 - Modelo de recomedación</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Máster Universitario en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
    * [1.1 - ¿Qué es LightFM?](#section11)
    * [1.2 - Motivo de la elección](#section12)
* [2. Creación del dataset en formato LightFM](#section2)
    * [2.1 - Interacciones playlists/pistas](#section21)
    * [2.2 - Características de las playlists](#section22)
    * [2.3 - Características de las pistas](#section23)
    * [2.4 - Formato y almacenamiento](#section24)
* [3. Creación y entrenamiento del modelo LightFM](#section3)
* [4. Resultados](#section4)
    * [4.1 - Definición de métricas](#section41)
    * [4.2 - Métricas obtenidas](#section42)
    * [4.3 - Resultados de nuestro modelo sobre diferentes playlist](#section43)
* [5. Información adicional obtenida del modelo](#section5)

<br>

---

In [1]:
import csv
import numpy as np
import os
import pandas as pd
import pickle
import joblib
import scipy.sparse as sp


from lightfm.data import Dataset
from tqdm.notebook import tqdm as tqdm_nb

c:\pythonenv\tfm\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
# Variables globales

# Directorio empleado para guardar/leer los datos generados
MPD_CSV_PATH = 'MPD_CSV'

ALBUMS_FILE = os.path.join(MPD_CSV_PATH,'mpd.albums.csv')
ALBUMS_RELEASE_FILE = os.path.join(MPD_CSV_PATH, 'mpd.albums-releasedate.csv')
ARTISTS_FILE = os.path.join(MPD_CSV_PATH,'mpd.artists.csv')
ARTISTS_GENRES_FILE = os.path.join(MPD_CSV_PATH, 'mpd.artists-genres.csv')
TRACKS_FILE = os.path.join(MPD_CSV_PATH,'mpd.tracks.csv')
PLSTRS_FILE = os.path.join(MPD_CSV_PATH,'mpd.pls-tracks.csv')
PLSINFO_FILE = os.path.join(MPD_CSV_PATH,'mpd.playlists-info.csv')
GENRES_FILE = os.path.join(MPD_CSV_PATH,'mpd.genres.csv')
PLSTESTINFO_FILE = os.path.join(MPD_CSV_PATH,'mpd.playlists-info-test.csv')

TFIDF_DATA_FILE = os.path.join(MPD_CSV_PATH,'mpd.tfidf-data.pkl')

# Directorio empleado para guardar/leer el modelo LightFM y sus datos
MODEL_FILES_PATH = "model"
MODEL_FILE = os.path.join(MODEL_FILES_PATH, 'model.pkl')
MODEL_DATA_FILE = os.path.join(MODEL_FILES_PATH, 'model_data.pkl')

In [3]:
def get_plsinfo():
    file = open(PLSINFO_FILE, "r", encoding='utf8')
    return csv.DictReader((line for line in file))

def get_plstestinfo():
    file = open(PLSTESTINFO_FILE, "r", encoding='utf8')
    return csv.DictReader((line for line in file))

def get_plstrs():
    file = open(PLSTRS_FILE, "r")
    return csv.DictReader((line for line in file))

def get_artists():
    file = open(ARTISTS_FILE, "r", encoding='utf8')
    return csv.DictReader((line for line in file))

def get_tracks():
    file = open(TRACKS_FILE, "r", encoding='utf8')
    return csv.DictReader((line for line in file))

def get_albums():
    file = open(ALBUMS_FILE, "r", encoding='utf8')
    return csv.DictReader((line for line in file))

<br>

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

En esta libreta vamos a preparar los datos que hemos obtenido para entrenar el modelo de *LightFM*, el cual usaremos para crear nuestro servicio de recomendación de playlists. Una vez entrenado el modelo de recomendación, estudiaremos los resultados que obtiene nuestro modelo y, a parte de las predicciones que realice, veremos qué información adicional puede aportarnos.

<br>

---

<br>

<a id="section11"></a>
### <font color="#92002A">1.1 - ¿Qué es *LightFM*?</font>
<br>

[*LightFM*](https://making.lyst.com/lightfm/docs/index.html) es una implementación en *Python* de una serie de algoritmos de recomendación para la retroalimentación implícita y explícita, incluida la implementación eficiente de las funciones de pérdida BPR y WARP. Es fácil de usar, rápido (a través de la estimación de modelos multiproceso) y produce resultados de alta calidad.

También permite incorporar metadatos de elementos y usuarios en los algoritmos tradicionales de factorización matricial, representando a cada usuario y elemento como la suma de las representaciones latentes de sus características, permitiendo así que las recomendaciones se generalicen a nuevos elementos (a través de las características del elemento) y a nuevos usuarios (a través de las características del usuario).

<br>

<br>

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
    <strong>Importante</strong>: <i>LightFM</i> hace uso de la librería <i>OpenMP</i>. En el caso de <i>Windows</i> y <i>macOS</i> no podremos emplear más de un hilo para realizar diferentes tareas (como la de entrenamiento), por lo que en caso de requerir el uso de múltiples hilos es necesario emplear un sistema operativo <i>Linux</i>.
</div>

<br>

---


<br>


<a id="section12"></a>
### <font color="#92002A">1.2 - Motivo de la elección</font>

<br>

La razón por la cual hemos optado por la librería *LightFM* es el uso de un sistema de recomendación hibrido, el cual resulta de combinar el filtrado colaborativo y el filtrado basado en contenido.

Otra de las razones por las cuales hemos optado por ella, es el problema del *arranque en frio*. En nuestro proyecto, debemos ser capaces de realizar una recomendación basada en las características del usuario y sin tener ninguna valoración previa de éste.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>


<a id="section2"></a>
## <font color="#92002A">2 - Creación del dataset en formato LightFM</font>

<br>

Para poder entrenar nuestro modelo, debemos convertir los datos que tenemos al formato de entrada que admite *LightFM*. Lo que necesitamos es crear las matrices de interacción entre playlists y pistas, junto a sus correspondientes matrices de características.

<br>

*LightFM* proporciona una clase dentro del módulo [`lightfm.data`](https://making.lyst.com/lightfm/docs/lightfm.data.html) llamada `Dataset`, con la que vamos a transformar el conjunto de datos que tenemos al formato admitido.

<br>

<div class="alert alert-info">

<i class="fa fa-info-circle" aria-hidden="true"></i>
__Nota__: *LightFM* trabaja con matrices de expansión, en formato de coordenadas o *CCO*. Para más información, consultar la documentación en [SciyPy](https://www.scipy.org/) disponible en el siguiente [enlace](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html#scipy.sparse.coo_matrix). 
</div>

<br>

---

<br>

<a id="section21"></a>
### <font color="#92002A">2.1 - Interacciones playlists/pistas</font>

<br>

Comenzamos definiendo el objeto de tipo `Dataset` que emplearemos para construir la matriz de interacción:

In [4]:
mpd_tracks = Dataset()

<br>

Incorporamos las playlist, que serán identificadas como usuarios, y las pistas del conjunto, empleadas como items:

In [5]:
mpd_tracks.fit(users=[x['pl_pid'] for x in get_plsinfo()] + 
                     [x['pl_pid'] for x in get_plstestinfo()],
               items=(x['track_pid'] for x in get_tracks()))

In [6]:
num_pls, num_trs = mpd_tracks.interactions_shape()
print('num_pls: {}, num_trs {}.'.format(num_pls, num_trs))

num_pls: 1010000, num_trs 2262292.


<br>

Una vez tenemos el objeto `mpd_tracks` definido, procedemos a crear la matriz de interacción:

In [7]:
track_interactions, _ = mpd_tracks.build_interactions(((x['pl_pid'], x['track_pid']))
                                                       for x in get_plstrs())
print(repr(track_interactions))

<1010000x2262292 sparse matrix of type '<class 'numpy.int32'>'
	with 66627428 stored elements in COOrdinate format>


---

<br>

<a id="section22"></a>
### <font color="#92002A">2.2 - Características de las playlists</font>

<br>


Una vez tenemos la matriz de interacción de playlists/pistas, recuperamos las características de las playlists. Dicha información se encuentra almacenada en un fichero *pickle*, que creamos previamente, y que contiene un diccionario con los nombres de las características y la matriz resultante de aplicar la técnica *TF-IDF*:

In [8]:
with open(TFIDF_DATA_FILE, "rb") as read_file:
    tfidf_data_dict = pickle.load(read_file)

<br>

<br>

<br>

¡¡¡ EXPLICAR MATRIZ IDENTIDAD !!!

<br>

<br>

<br>

In [9]:
playlist_features_matrix = sp.hstack([sp.eye(tfidf_data_dict['matrix'].shape[0]),
                                     tfidf_data_dict['matrix']])

playlist_features_matrix

<1010000x1020511 sparse matrix of type '<class 'numpy.float64'>'
	with 2463332 stored elements in COOrdinate format>

---

<br>

<a id="section23"></a>
### <font color="#92002A">2.3 - Características de las pistas</font>

<br>

Para crear las características de las pistas, vamos a crear una matriz para cada uno de los siguientes casos:

* Matriz de características para pistas/álbumes
* Matriz de características para pistas/artistas
* Matriz de pistas/año de lanzamiento.
* Matriz de pistas/géneros.

<br>

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
    <strong>Importante</strong>: Aunque hemos obtenido de <i>Spotify</i> las características de audio para cada pista, no nos ha sido posible añadir dicha información para entrenar el modelo. Tras varias pruebas, hemos podido comprobar que el proceso de entrenamiento era más costoso / le resultaba más complicado aprender con los datos proporcionados.
</div>

<br>

### <font color="#92002A">Características para pistas/álbumes</font>

<br>

Estas características servirán para asociar las pistas a sus correspondientes álbumes. Comenzamos recuperando la información de los álbumes y de las pistas:

In [10]:
df_albums = pd.read_csv(ALBUMS_FILE)
df_tracks = pd.read_csv(TRACKS_FILE)

<br>

A continuación, creamos una matriz de expansión en formato diccionario (también conocida como _DoK_, _Dictionary of Keys_), donde las filas serán las pistas y las columnas los álbumes. El valor de la celda se establecerá a 1 en caso de que la pista pertenezca al álbum.

In [11]:
album_features_matrix = sp.dok_matrix((len(df_tracks),len(df_albums)), dtype=int)

for _ ,row in tqdm_nb(df_tracks.iterrows(), total=len(df_tracks)):
    album_features_matrix[row['track_pid'], row['album_pid']] = 1

  0%|          | 0/2262292 [00:00<?, ?it/s]

<br>

Una vez creada la matriz, la convertimos a formato *COO* y almacenamos en una lista los identificadores de *Spotify* a los cuales pertenece cada columna:

In [12]:
album_features_matrix = album_features_matrix.tocoo()
album_features_names = df_albums['album_id'].to_list()

album_features_matrix

<2262292x734684 sparse matrix of type '<class 'numpy.int32'>'
	with 2262292 stored elements in COOrdinate format>

<br>

### <font color="#92002A">Características para pistas/artistas</font>

<br>

De igual manera que acabamos de hacer para el caso de los álbumes, repetiremos el mismo proceso para los artistas:

In [13]:
df_artists = pd.read_csv(ARTISTS_FILE)

In [14]:
artist_features_matrix = sp.dok_matrix((len(df_tracks),len(df_artists)), dtype=int)

for _ ,row in tqdm_nb(df_tracks.iterrows(), total=len(df_tracks)):
    artist_features_matrix[row['track_pid'], row['artist_pid']] = 1

artist_features_matrix = artist_features_matrix.tocoo()
artist_features_names = df_artists['artist_id'].to_list()

artist_features_matrix

  0%|          | 0/2262292 [00:00<?, ?it/s]

<2262292x295860 sparse matrix of type '<class 'numpy.int32'>'
	with 2262292 stored elements in COOrdinate format>

<br>

### <font color="#92002A">Características para pistas/año de lanzamiento</font>

<br>

A continuación, leemos el fichero que contiene las fechas de lanzamiento de los álbumes y nos quedamos únicamente con el año:

In [15]:
df_releases = pd.read_csv(ALBUMS_RELEASE_FILE)

In [16]:
def get_release_year(release):
    split_text = str(release).split('-')
    
    if release == None:
        return None
    else:
        return split_text[0]

In [17]:
df_releases['release_year'] = df_releases['release_date'].apply(get_release_year)
df_releases.drop(columns=['release_date'], inplace=True)
df_releases = pd.merge(df_tracks, df_releases, how='left', on='album_id')[['track_pid', 'release_year']]

<br>

Con la ayuda de *pandas*, creamos la matriz de interacción entre pistas y año de lanzamiento:

In [18]:
df_releases = df_releases.set_index('track_pid')['release_year'].str.get_dummies().astype(int).reset_index()
df_releases.set_index('track_pid', inplace=True)

In [19]:
df_releases.head()

,0000,0001,0013,1197,1885,1889,1899,1900,1901,1905,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
track_pid,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<br>

Como podemos ver en los nombres de las columnas del dataframe, existen años que son incorrectos: *0000*, *0001*, *0013* y *1197*. Los eliminamos y procedemos a transformar los datos a una matriz de expansión:

In [20]:
df_releases.drop(columns=['0000','0001','0013','1197'], inplace=True)

In [21]:
year_features_matrix = sp.coo_matrix(df_releases.values)
year_features_names = df_releases.columns.to_list()

year_features_matrix

<2262292x116 sparse matrix of type '<class 'numpy.int32'>'
	with 2261642 stored elements in COOrdinate format>

<br>

### <font color="#92002A">Características para pistas/géneros</font>

<br>

Por último, nos falta incorporar a las características la información de los géneros a los que pertenece el artista de la pista. Cargamos en un dataframe los géneros a los que pertenece cada artista y sustituimos el identificador de *Spotify* por el nuestro:

In [22]:
df_artists_genres = pd.read_csv(ARTISTS_GENRES_FILE)
df_artists_genres = pd.merge(df_artists, df_artists_genres, how='left', on='artist_id')[['artist_pid', 'genres']]

<br>

A continuación, asociamos los géneros del artista de la pista a cada una de ellas:

In [23]:
df_tracks_genres = df_tracks[['track_pid', 'artist_pid']]
df_tracks_genres = pd.merge(df_tracks_genres, df_artists_genres, how='left', on='artist_pid')
df_tracks_genres.drop(columns=['artist_pid'], inplace=True)

<br>

Como los géneros estaban en un string separados por el carácter `\`, convertimos el valor de columna `genres` a una lista:

In [24]:
df_tracks_genres['genres'] = df_tracks_genres['genres'].astype(str)
df_tracks_genres['genres'] = df_tracks_genres['genres'].apply(lambda x: x.split('|'))

<br>

Mediante la función `explode` de los dataframes, separamos en varias filas los valores de los géneros que hay en la lista para cada pista:

In [25]:
df_tracks_genres.head(3)

,track_pid,genres
0,0,"[dance pop, hip hop, hip pop, pop, pop rap, r&..."
1,1,"[dance pop, pop, post-teen pop]"
2,2,"[dance pop, pop, post-teen pop, r&b]"


In [26]:
df_tracks_genres = df_tracks_genres.explode('genres')
df_tracks_genres.rename(columns={"genres": "genre_name"}, inplace=True)

# Eliminamos los valores 'nan', que fueron convertidos a string en el paso anterior
df_tracks_genres = df_tracks_genres[df_tracks_genres['genre_name'] != 'nan']

In [27]:
df_tracks_genres.head()

,track_pid,genre_name
0,0,dance pop
0,0,hip hop
0,0,hip pop
0,0,pop
0,0,pop rap


<br>

Antes de continuar, vamos a crear un nuevo _dataframe_ que contendrá todos los géneros disponibles y les establecerá un identificador:

In [28]:
df_genres = pd.DataFrame()
df_genres['genre_name'] = list(df_tracks_genres['genre_name'].unique())
df_genres.index.name = 'genre_pid'

# Lo guardamos en un fichero CSV
df_genres.to_csv(GENRES_FILE)

<br>

Ahora sólo nos falta establecer los identificadores de los géneros a las pistas. Para ello, vamos a crear una nueva matriz de expansión en formato _DoK_ e iteramos por las filas del `dataframe` para establecer a 1 aquellos géneros a los que pertenece la pista:

In [29]:
# Asociamos los identificadores a los géneros correspondientes en el dataframe
df_tracks_genres = pd.merge(df_tracks_genres,df_genres.reset_index(), how='left', on='genre_name')

genre_features_matrix = sp.dok_matrix((len(df_tracks),len(df_genres)), dtype=int)

for _ ,row in tqdm_nb(df_tracks_genres.iterrows(), total=len(df_tracks_genres)):
    genre_features_matrix[row['track_pid'], row['genre_pid']] = 1

  0%|          | 0/6197748 [00:00<?, ?it/s]

<br>

Una vez creada la matriz, la convertimos a formato *COO* y almacenamos en una lista los nombres de los géneros a los que pertenece cada columna:

In [30]:
genre_features_matrix = genre_features_matrix.tocoo()
genre_features_names = df_genres['genre_name'].to_list()

genre_features_matrix

<2262292x5326 sparse matrix of type '<class 'numpy.int32'>'
	with 6197748 stored elements in COOrdinate format>

---

<br>

<a id="section24"></a>
### <font color="#92002A">2.4 - Formato y almacenamiento</font>

<br>

Una vez que tenemos todas las matrices correspondientes a las características de las pistas, los unimos en una única matriz:

<br>

<br>

<br>

¡¡¡ EXPLICAR MATRIZ IDENTIDAD !!!

<br>

<br>

<br>

In [31]:
#Need to hstack user_features
track_features_matrix = sp.hstack([sp.eye(len(df_tracks)),
                                  album_features_matrix,
                                  artist_features_matrix,
                                  year_features_matrix,
                                  genre_features_matrix])

track_features_matrix

<2262292x3298278 sparse matrix of type '<class 'numpy.float64'>'
	with 15246266 stored elements in COOrdinate format>

<br>

También vamos a crear una lista con todos los nombres de las características, añadiendo un prefijo para identificar el tipo de característica:

In [32]:
track_features_names = (['track:' + tr for tr in df_tracks['track_id'].to_list()] +
                        ['album:' + alb for alb in album_features_names] +
                        ['artist:' + art for art in artist_features_names] +
                        ['year:' + str(year) for year in year_features_names] +
                        ['genre:' + genre for genre in genre_features_names])

<br>

Con todos los datos necesarios para crear el modelo de recomendación, creamos un diccionario que contiene toda la información necesaria para el entrenamiento:

In [33]:
plstrs_data_lfm_dict = dict()

plstrs_data_lfm_dict['playlist_names'] = ([x['name'] for x in get_plsinfo()] +  
                                          [x['name'] for x in get_plstestinfo()])
plstrs_data_lfm_dict['playlist_interactions'] = track_interactions
plstrs_data_lfm_dict['playlist_features'] = playlist_features_matrix
plstrs_data_lfm_dict['playlist_features_names'] = (['name:' + x['name'] for x in get_plsinfo()] +  
                                               ['name:' + x['name'] for x in get_plstestinfo()] +
                                               ['feat:' + feat_name for feat_name in tfidf_data_dict['features_list']])

plstrs_data_lfm_dict['track_features'] = track_features_matrix
plstrs_data_lfm_dict['track_features_names'] = track_features_names

<br>

Por último, lo almacenamos en un fichero *pickle*:

In [34]:
if not os.path.exists(MODEL_FILES_PATH):
    os.makedirs(MODEL_FILES_PATH)

with open(MODEL_DATA_FILE, "wb") as write_file:
    pickle.dump(plstrs_data_lfm_dict, write_file, protocol=pickle.HIGHEST_PROTOCOL)

<br>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#92002A">3 - Creación y entrenamiento del modelo LightFM</font>
<br>



<br>


Para la creación del modelo, y al disponer de una gran cantidad de datos con los que realizar el entrenamiento, hemos decidido crear una [máquina virtual en *Microsoft Azure*](https://azure.microsoft.com/es-es/services/virtual-machines/) con las siguientes características:
    
<br>

| Serie | Descripción        | Procesador                 | Instancia | vCPU | RAM    | Almacenamiento    | Sistema operativo       |
| ----- | ------------------ | -------------------------- | --------- | ---- | ------ | ------------------| ----------------------- |
| Fsv2  | Proceso optimizado | Intel Xeon Platinum 8272CL | F32s v2   | 32   | 64 GB  | 256 GB (Temporal) | Ubuntu Server 20.04 LTS |

<font size=1>* Más información de la serie *Fsv2* en el siguiente [enlace](https://docs.microsoft.com/es-es/azure/virtual-machines/fsv2-series).</font>

<br>

Una vez que la máquina virtual esta creada e iniciada, procedemos a subir el fichero con los datos necesarios para el entrenamiento (`model_data.pkl`) y un script que se encargará de realizar el proceso:

```python
import numpy as np
import os
import pickle
import sys
import time

from lightfm import LightFM
from modules.TelegramBot import telegram_bot_sendtext


PROCESS_NANE = "TrainVM"
SEED = 1


def train_model(model_name, model_data_file, model_storage_path, num_epochs, num_threads):
    if not os.path.isdir(model_storage_path):
        os.mkdir(model_storage_path)
    model_file_path = os.path.join(model_storage_path,f"{model_name}.pkl")

    with open(model_data_file, "rb") as read_file:
        mpd_lfm_dict = pickle.load(read_file)

    interactions_weights = None
    if "playlist_interactions_weights" in mpd_lfm_dict.keys():
        interactions_weights = mpd_lfm_dict['playlist_interactions_weights']

        
    model = LightFM(loss='warp', no_components=200, max_sampled=30, random_state=SEED)

    ## Entrenamiento del modelo ##
    start_time = time.time()
    train_error = False
    try:
        model.fit(interactions=mpd_lfm_dict['playlist_interactions'],
                sample_weight=interactions_weights, 
                item_features=mpd_lfm_dict['track_features'], 
                user_features=mpd_lfm_dict['playlist_features'],
                epochs=num_epochs, num_threads=num_threads, 
                verbose=True)
        
        duration = (time.time() - start_time)
        duration = np.round(duration/60,2)

        message = f"Entrenamiento completado. Tiempo empleado: {duration} min."
        print(message)
        telegram_bot_sendtext(message, PROCESS_NANE) 
    except Exception as e:
        message = f"ERROR: Se ha producido un error durante el proceso de entrenamiento ({str(e)})"
        telegram_bot_sendtext(message, PROCESS_NANE)   
        print(message)
        return        
    
    ## Almacenamiento del modelo ##
    try:
        print("Almacenando modelo...")
        telegram_bot_sendtext("Almacenando modelo", PROCESS_NANE)
        joblib.dump(model, open(model_file_path, 'wb'))
    except Exception as e:
        message = f"ERROR: El modelo no ha podido ser almacenado ({str(e)})"
        telegram_bot_sendtext(message, PROCESS_NANE)
        print(message)
        return

    ## Comprobación del modelo almacenado ##
    try:
        joblib.load(open(model_file_path, 'rb'))
        print('Modelo almacendo correctamente')
        telegram_bot_sendtext('Modelo almacenado correctamente', PROCESS_NANE)
    except Exception as e:
        message = f"ERROR: El modelo no se ha almacenado correctamente ({str(e)})"
        telegram_bot_sendtext(message, PROCESS_NANE)
        print(message)


if __name__ == "__main__":
    if len(sys.argv) > 4:
        try:
            model_name = sys.argv[1]
            model_data_file = sys.argv[2]
            model_storage_folder = sys.argv[3]
            num_epochs = int(sys.argv[4])

            if len(sys.argv) > 5:
                num_threads = int(sys.argv[5])
            else:
                num_threads = 1

            train_model(model_name, model_data_file, model_storage_folder, num_epochs, num_threads)

        except Exception as e:
            print(f"ERROR: {str(e)}")
    
    else:
        print("ERROR: Invalid number of arguments.\nRequired -> model_name, model_data_file, model_storage_folder, num_epochs")
```

<br>

Datos del entrenamiento del modelo:

* <u>Tiempo de ejecución</u>: 242 minutos (≈ 4 horas)
* <u>Fichero del modelo</u>: model/model.pkl
* <u>Tamaño del modelo</u>: 9,68 GB
* <u>Número de Epochs establecido</u>: 75

<br>

Con el modelo ya entrenado y almacenado, procedemos a descargarlo a nuestra máquina.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section4"></a>
## <font color="#92002A">4 - Resultados</font>

<br>

Intentando evaluar nuestro modelo con las métricas que ofrece *LightFM*, nos encontramos con que, tras más de 4-5 horas de ejecución, el proceso no terminaba y no queríamos realizar un gasto elevado en *Microsoft Azure* (puesto que disponíamos de un saldo limitado). Por dicho motivo, decidimos detener el proceso de evaluación del modelo para obtener las métricas.

Recientemente se ha liberado el conjunto *Million Playlist Dataset*, construido por *Spotify* para la competición [*RecSys Challenge* (edición del 2018)](https://recsys.acm.org/recsys18/challenge/). Este conjunto puede ser obtenido en el portal [AIcrowd](https://www.aicrowd.com/), pudiendo participar en el reto abierto *[Spotify Million Playlist Dataset Challenge](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge)*.

Como en el reto que está disponible en *AIcrowd*, tras enviar los resultados de las predicciones sobre el conjunto de test, nos ofrece una serie de métricas y podemos comparar los resultados con los de otros participantes, hemos decidido repetir todos los pasos que hemos realizado hasta ahora empleando dicho conjunto (ya que el conjunto que creamos en su momento está basado en el *Million Playlist Dataset*).

<br>

---

<br>

<a id="section41"></a>
### <font color="#92002A">4.1 - Definición de las métricas</font>

<br>


Los envíos realizados al reto se evalúan utilizando las siguientes métricas. Todas las métricas son evaluadas tanto a nivel de pista (coincidencia exacta de pista) como a nivel de artista (cualquier pista del mismo artista es una coincidencia).

A continuación, denotamos el conjunto de pistas reales (las que debemos recomendar) por *`G`* y la lista ordenada de pistas recomendadas por *`R`*. El tamaño de un conjunto o lista se denota por `|·|`, y usamos *from:to-subcripts* para indexar una lista.

<br><br>

<strong><font color="#92002A" size=4>R-Precision</font></strong>

*R-precision* es el número de pistas relevantes recuperadas dividido por el número de pistas relevantes conocidas (es decir, la cantidad de pistas retenidas):

<br><center><font size=4><i>R-Precision</i> = <font size=5> $\frac{|G\cap R_{1:|G|}|}{|G|} $ </font></font></center><br>

La métrica se promedia en todas las listas de reproducción del conjunto de test (o desafío). Esta métrica recompensa el número total de pistas relevantes recuperadas (independientemente del orden).

<br><br>

<strong><font color="#92002A" size=4>Normalized Discounted Cumulative Gain (NDCG)</font></strong>

_Discounted Cumulative Gain_, o _DCG_, mide la calidad de clasificación de las pistas recomendadas y aumenta cuando las pistas relevantes se colocan más arriba en la lista. La DCG normalizada (_NDCG_) se determina calculando la DCG y dividiéndola por la DCG ideal en la que las pistas recomendadas están perfectamente clasificadas:

<br><center><font size=4><i>DCG</i> = $rel_{1} + \sum_{i=2}^{|R|} \frac{rel_{i}}{log_{2}i} $ </font></center><br>

El DCG ideal o _IDCG_ es, en nuestro caso, igual a:

<br><center><font size=4><i>IDCG</i> = $1 + \sum_{i=2}^{|G\cap R|} \frac{1}{log_{2}i} $ </font></center><br>

Si el tamaño de la intersección establecida de *`G`* y *`R`* es vacío, entonces el IDCG es igual a 0. La métrica NDCG ahora se calcula como:

<br><center><font size=4><i>NDCG</i> = $\frac{DCG}{IDCG} $ </font></center><br>


<br><br>

<strong><font color="#92002A" size=4>Recommended song clicks</font></strong>

_Recommended song clicks_ (o canciones recomendadas) es una función de Spotify que, dado un conjunto de pistas en una lista de reproducción, recomienda 10 pistas para agregar a la lista de reproducción. La lista se puede actualizar para producir 10 pistas más. Los clics de canciones recomendadas son la cantidad de actualizaciones necesarias antes de que se encuentre una pista relevante. Se calcula de la siguiente manera:

<br><center><font size=4><i>clicks</i> = $\left[ \frac{arg \ min_{i}\{R_{i}:R_{i} \in G\} - 1}{10} \right] $ </font></center><br>

Si la métrica no existe (es decir, si no hay pistas relevantes en *`R`*, se elige un valor de 51 (que es 1 mayor que el número máximo si es posible hacer clic)

<br>

---

<br>

<a id="section42"></a>
### <font color="#92002A">4.2 - Métricas obtenidas</font>

<br>

Una vez que hemos realizado las predicciones para las playlist del reto y las hemos enviado al sistema de evaluación, hemos obtenido los siguientes resultados:

* <u>R-prec</u>: 0,220
* <u>NDCG</u>: 0,341
* <u>Recommended Song Clicks</u>: 2,212

<br>

Si comparamos los resultados con los del primer clasificado (a fecha 21/08/2021):

<br>

| # | R-prec | NDCG  | Recommended Song Clicks | Nota               |
| - |------- | ----- | ----------------------- | ------------------ |
| 1 | 0,220	 | 0,386 | 1,932                   | Primer clasificado |
| - | ------ | ----- | ----------------------- | ------------------
| 5 | 0,187	 | 0,341 | 2,212                   | Resultado obtenido |

<br>

podemos ver que los resultados obtenidos por nuestro modelo son bastante prometedores.

<br>

---

<br>

<a id="section43"></a>
### <font color="#92002A">4.3 - Resultados de nuestro modelo sobre diferentes playlist</font>

<br>

Por último, vamos a elegir 3 playlists del conjunto de test y comprobaremos los resultados que ofrece nuestro modelo. Las playlists elegidas, junto a sus características, son las siguientes:

| # | model_id | pl_pid  | Contiene título | Pistas disponibles | Nombre de la playlist | Características                        |
| - | -------  | ------- | --------------- | ------------------ | --------------------- | -------------------------------------- |
| 1 | 1000000  | 1000002 | Sí              | 0                  | spanish playlist      | Música en español                      |
| 2 | 1003271  | 1008130 | Sí              | 10                 | throwback songs       | Música pop internacional (2007-2015)   |
| 3 | 1004832  | 1013678 | No              | 10                 | -                     | Música urbana (R&B, Hip-Hop, Rap, ...) |

<br>

Antes de comenzar, vamos a crear un *dataframe* con la información de las pistas (nombre de la pista, nombre del álbum, nombre del artista y fecha de lanzamiento), vamos a leer cuáles son las pistas disponibles para las playlist de las que tenemos dicha información y cargar el modelo que hemos entrenado:

In [35]:
# Dataframe con las pistas de las playlist 1008130, 1013678
df_plstrs = pd.read_csv(PLSTRS_FILE, index_col=0).loc[[1008130, 1013678]]

# Dataframe con la fecha de lanzamiento de los álbumes
df_album_release = pd.read_csv(ALBUMS_RELEASE_FILE)

In [36]:
# Dataframe con la información completa de las pistas
df_tracks_info = pd.merge(df_tracks[['track_pid', 'track_name', 'album_pid', 'artist_pid']], 
                          df_albums[['album_pid', 'album_name', 'album_id']], how='left', on='album_pid')
df_tracks_info = pd.merge(df_tracks_info, df_artists[['artist_pid', 'artist_name']], how='left', on='artist_pid')
df_tracks_info = pd.merge(df_tracks_info, df_album_release, how='left', on='album_id')
df_tracks_info.drop(columns=['album_pid', 'artist_pid', 'album_id'], inplace=True)

df_tracks_info.head()

,track_pid,track_name,album_name,artist_name,release_date
0,0,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Missy Elliott,2005-07-04
1,1,Toxic,In The Zone,Britney Spears,2003-11-13
2,2,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Beyoncé,2003-06-23
3,3,Rock Your Body,Justified,Justin Timberlake,2002-11-04
4,4,It Wasn't Me,Hot Shot,Shaggy,2000


In [39]:
model = joblib.load(open(MODEL_FILE, 'rb'))
model_data = joblib.load(open(MODEL_DATA_FILE, 'rb'))

<br>

También vamos a definir una función con la que realizaremos las predicciones sobre las playlists indicadas:

In [40]:
def make_user_recomm(user_id, model, u_features, i_features, num_items, n_recs=1000, n_threads=1):
    if n_recs > 1000:
        n_recs = 1000
        
    scores = model.predict(user_id, np.arange(num_items), user_features=u_features, 
                           item_features=i_features, num_threads=n_threads)
    items = list(np.argsort(scores)[::-1])[:n_recs]
    
    result = []    
    for item_id in items:
        result.append({'item_id' : int(item_id), 'score' : float(scores[item_id])})
        
    return result

<br><br>

<strong><font color="#92002A" size=4>Playlist 1000002 - "<i>spanish playlist</i>"</font></strong>

<br>

Obtenemos las 16 predicciones con la puntuación más alta:

In [41]:
pl1_predictions = make_user_recomm(1000000, model, 
                                   model_data['playlist_features'],
                                   model_data['track_features'],
                                   len(df_tracks), 16)

<br>

Guardamos los identificadores de las pistas y mostramos los resultados:

In [42]:
pl1_recommended_pids = [x['item_id'] for x in pl1_predictions]

df_tracks_info.loc[pl1_recommended_pids]

,track_pid,track_name,album_name,artist_name,release_date
14272,14272,Vivir Mi Vida,3.0,Marc Anthony,2013-07-23
5313,5313,El Perdón,Fénix,Nicky Jam,2017-01-20
5411,5411,Bailando - Spanish Version,SEX AND LOVE,Enrique Iglesias,2014-01-01
5397,5397,Hasta el Amanecer,Fénix,Nicky Jam,2017-01-20
6164,6164,Danza Kuduro,Meet The Orphans,Don Omar,2010-01-01
3764,3764,Suavemente,Suavemente,Elvis Crespo,1998-02-27
1330,1330,Despacito (Featuring Daddy Yankee),Despacito (Featuring Daddy Yankee),Luis Fonsi,2017-01-13
3780,3780,La Camisa Negra,Mi Sangre,Juanes,2004-01-01
3806,3806,Corazon Sin Cara,Prince Royce,Prince Royce,2010
20870,20870,Darte un Beso,Soy el Mismo,Prince Royce,2013-10-08


<br>

Como podemos apreciar en las recomendaciones para la playlist "*spanish playlist*", las pistas se identifican perfectamente con el título. Todas ellas pertenecen a artistas que cantan en español (ya sean de España o Latinoamérica).

<br>

<strong><font color="#92002A" size=4>Playlist 1008130 - "<i>throwback songs</i>"</font></strong>

<br>

Para esta playlist, a parte del título, si disponemos de algunas pistas. Procedemos a mostrarlas:

In [43]:
tr_ids = df_plstrs.loc[1008130]['track_pid'].to_list()

df_tracks_info.loc[tr_ids]

,track_pid,track_name,album_name,artist_name,release_date
936,936,Mirrors,The 20/20 Experience (Deluxe Version),Justin Timberlake,2013-03-15
5684,5684,Blame,Motion,Calvin Harris,2014-10-31
43,43,Baby,My Worlds,Justin Bieber,2010-01-01
4691,4691,Apologize,Dreaming Out Loud,OneRepublic,2007-01-01
19035,19035,Your Type,Emotion,Carly Rae Jepsen,2015-06-24
6974,6974,Explosions,Halcyon,Ellie Goulding,2012-01-01
1021,1021,DJ Got Us Fallin' In Love,Raymond v Raymond (Deluxe Edition),Usher,2010-03-30
41,41,Somebody To Love,My Worlds,Justin Bieber,2010-01-01
60455,60455,Don't Dream It's Over (Glee Cast Version),"Glee: The Music, The Complete Season Four",Glee Cast,2014-01-14
3058,3058,You'll Always Find Your Way Back Home,Hannah Montana The Movie,Hannah Montana,2009-01-01


<br>

Cargamos las 16 predicciones, con la puntuación más alta y las mostramos en una tabla:

In [44]:
pl2_predictions = make_user_recomm(1003271, model, 
                                   model_data['playlist_features'],
                                   model_data['track_features'],
                                   len(df_tracks), 16)

pl2_recommended_pids = [x['item_id'] for x in pl2_predictions]

df_tracks_info.loc[pl2_recommended_pids]

,track_pid,track_name,album_name,artist_name,release_date
41468,41468,"I Got Nerve - From ""Hannah Montana""/Soundtrack...",Hannah Montana,Hannah Montana,2006-01-01
29550,29550,Get'Cha Head In The Game,High School Musical,High School Musical Cast,2006-01-01
120797,120797,Need a Little Love,Hannah Montana Forever,Hannah Montana,2010-01-01
28,28,Whatcha Say,Jason Derulo,Jason Derulo,2010-02-24
15160,15160,Crank That (Soulja Boy),souljaboytellem.com,Soulja Boy,2007-01-01
41472,41472,Everyday,High School Musical 2,High School Musical Cast,2007-01-01
28697,28697,So Yesterday,Metamorphosis,Hilary Duff,2003-01-01
2654,2654,The Climb,The Time Of Our Lives,Miley Cyrus,2009-01-01
29556,29556,Let's Dance,Hannah Montana 2 / Meet Miley Cyrus,Miley Cyrus,2007-01-01
29543,29543,"Rockstar - Live from Arrowhead Pond, Anaheim, ...",Hannah Montana/Miley Cyrus: Best of Both World...,Hannah Montana,2008-01-01


<br>

Como podemos observar, los géneros, artistas y fechas de lanzamiento, se relacionan con las 10 pistas por las que estaba compuesta la playlist.

<br>

<br>

<strong><font color="#92002A" size=4>Playlist 1013678 (música de tipo urbano)</font></strong>

<br>

Para esta última playlist, no disponemos del título, pero si disponemos de algunas pistas. Procedemos a mostrarlas:

In [45]:
tr_ids = df_plstrs.loc[1013678]['track_pid'].to_list()

df_tracks_info.loc[tr_ids]

,track_pid,track_name,album_name,artist_name,release_date
13724,13724,Hard In Da Paint - Explicit Album Version,Flockaveli,Waka Flocka Flame,2010-10-01
2781,2781,No Hands (feat. Roscoe Dash and Wale) - Explic...,Flockaveli,Waka Flocka Flame,2010-10-01
12894,12894,Scholarship,Stay Trippy,Juicy J,2013-08-23
2771,2771,Wild for the Night,LONG.LIVE.A$AP (Deluxe Version),A$AP Rocky,2013
3155,3155,Work REMIX,Trap Lord,A$AP Ferg,2013-08-19
29240,29240,All Gold Everything - Remix,Don't Be S.A.F.E.,Trinidad James,2013-01-01
18504,18504,Believe It - feat. Rick Ross,Dreams and Nightmares,Meek Mill,2012-10-26
18284,18284,No Lie,Based On A T.R.U. Story,2 Chainz,2012-01-01
1373,1373,The Next Episode,2001,Dr. Dre,1999-11-16
111952,111952,Smokin & Drinkin,Old,Danny Brown,2013-10-08


<br>

Si realizamos las predicciones para esta playlist, obtenemos los siguientes resultados:

In [46]:
pl3_predictions = make_user_recomm(1004832 , model, 
                                   model_data['playlist_features'],
                                   model_data['track_features'],
                                   len(df_tracks), 16)

pl3_recommended_pids = [x['item_id'] for x in pl3_predictions]

df_tracks_info.loc[pl3_recommended_pids]

,track_pid,track_name,album_name,artist_name,release_date
11810,11810,Shabba,Trap Lord,A$AP Ferg,2013-08-19
3155,3155,Work REMIX,Trap Lord,A$AP Ferg,2013-08-19
11140,11140,Man Of The Year,Oxymoron,ScHoolboy Q,2014-01-01
21553,21553,All Gold Everything,Don't Be S.A.F.E.,Trinidad James,2013-01-01
529,529,m.A.A.d city,"good kid, m.A.A.d city",Kendrick Lamar,2012
3160,3160,Collard Greens,Oxymoron,ScHoolboy Q,2014-01-01
549,549,Hot N*gga,Hot N*gga,Bobby Shmurda,2014-07-25
8913,8913,All Me,Nothing Was The Same,Drake,2013-01-01
2770,2770,F**kin' Problems,LONG.LIVE.A$AP (Deluxe Version),A$AP Rocky,2013
11815,11815,Old English,Peacemaker,Salva,2016-07-01


<br>

Si observamos las predicciones mostradas en la tabla anterior, vemos que todas las pistas se corresponden con el tipo de música esperado. También cabe destacar que, para la mayoría de las psitas, su año de lanzamiento se situa entre 2013-2014, muy próximo a las pistas iniciales (comprendidas entre 2010 y 2013).

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section5"></a>
## <font color="#92002A">5 - Información adicional obtenida del modelo</font>
<br>

Como LightFM se basa en un modelo de recomendación híbrido, y para entrenarlo hemos utilizado algunas características para identificar las playlists y las pistas, podemos obtener características similares a una dada. Por ejemplo, si buscamos etiquetas similares a *latino* deberíamos obtener etiquetas que tengan el mismo significado o similar. Podemos obtener la misma información para artistas, álbumes y géneros. También podemos obtener aquellas pistas que son similares a una dada.

A continuación, mostramos que características similares ofrece nuestro modelo para la etiqueta *latino*, que en este caso se corresponde con una característica de las playlists:


<br>

In [47]:
def get_similar_user_tags(tag_id, model, N=100):
    # Define similarity as the cosine of the angle
    # between the tag latent vectors

    # Normalize the vectors to unit length
    tag_embeddings = (model.user_embeddings.T
                      / np.linalg.norm(model.user_embeddings, axis=1)).T

    query_embedding = tag_embeddings[tag_id]
    similarity = np.dot(tag_embeddings, query_embedding)
    most_similar = np.argsort(-similarity)[1:N+1]
    
    result = []
    for user_id in most_similar:
        result.append({'user_tag_id' : int(user_id), 'score' : float(similarity[user_id])})

    return result

In [48]:
# Obtenemos el identificador de la etiqueta:
model_data['playlist_features_names'].index('feat:latino')

1015187

In [49]:
# Buscamos las etiquetas relacionadas
results = get_similar_user_tags(1015187, model, N=16)

tags_ids = [x['user_tag_id'] for x in results]

[model_data['playlist_features_names'][tag_id] for tag_id in tags_ids]

['feat:latina',
 'feat:español',
 'feat:spanish',
 'feat:latin',
 'feat:espanol',
 'feat:hispanic',
 'feat:espanish',
 'feat:baila',
 'feat:bailamos',
 'feat:reggaeton',
 'feat:reggeton',
 'name:🇲🇽',
 'feat:reggaton',
 'feat:regueton',
 'feat:bailando',
 'name:🇲🇽🇲🇽🇲🇽']

<br>

Como podemos ver en los resultados, hemos obtenido etiquetas similares o relacionadas con *latino* al igual que se nos ofrecen títulos de playlist comunes a esas etiquetas (en este caso los dos títulos obtenidos equivalen a *México*).

<br>

Para terminar, vamos a buscar características similares a otra dada, pero pertenecientes a las pitas. En este caso vamos a elegir un cantante, *Mariah Carey* , para ver cuáles son los cantantes similares según nuestro modelo:

In [50]:
def get_similar_item_tags(tag_id, model, N=100):
    # Define similarity as the cosine of the angle
    # between the tag latent vectors

    # Normalize the vectors to unit length
    tag_embeddings = (model.item_embeddings.T
                      / np.linalg.norm(model.item_embeddings, axis=1)).T

    query_embedding = tag_embeddings[tag_id]
    similarity = np.dot(tag_embeddings, query_embedding)
    most_similar = np.argsort(-similarity)[1:N+1]
    
    result = []
    for item_id in most_similar:
        result.append({'item_tag_id' : int(item_id), 'score' : float(similarity[item_id])})

    return result

In [51]:
mariahcarey_tag = model_data['track_features_names'].index('artist:4iHNK0tOyZPYnBU7nGAgpQ')
mariahcarey_tag

2997110

In [52]:
results = get_similar_item_tags(2997110, model, N=16)

tags_ids = [x['item_tag_id'] for x in results]
tag_list = [model_data['track_features_names'][tag_id] for tag_id in tags_ids]
tag_list

['track:4LU9Gg0njNhCS2QbUS4xut',
 'artist:5lKZWd6HiSCLfnDGrq9RAm',
 'album:43iBTEWECK7hSnE0p6GgNo',
 'artist:1Y8cdNmUJH7yBTd9yOvr5i',
 'album:7JXUd8N7OXYQz981E5Jaq2',
 'artist:6XpaIBNiVzIetEPCWDvAFP',
 'album:1ibYM4abQtSVQFQWvDSo4J',
 'artist:05oH07COxkXKIMt6mIPRee',
 'artist:5rkVyNGXEgeUqKkB5ccK83',
 'track:6t9qYVZI8QDRoQzOCoaVsI',
 'artist:63wjoROpeh5f11Qm93UiJ1',
 'artist:0TImkz4nPqjegtVSMZnMRq',
 'album:1iSTXHBhLc9ImaqyvVZGft',
 'artist:4eAOcbAXIF4BmbN6E1QIlw',
 'album:3MJHoQUI828kmB6IpjejbW',
 'artist:1l7ZsJRRS8wlW3WfJfPfNS']

<br>

En este caso, como todas las etiquetas están dentro de un único conjunto, también hemos obtenido pistas y álbumes relacionados con *Mariah Carey*. Procedemos a mostrar únicamente los artistas relacionados:

In [53]:
artists_ids = [x.split(':')[1] for x in tag_list if 'artist:' in x]
artists_ids

['5lKZWd6HiSCLfnDGrq9RAm',
 '1Y8cdNmUJH7yBTd9yOvr5i',
 '6XpaIBNiVzIetEPCWDvAFP',
 '05oH07COxkXKIMt6mIPRee',
 '5rkVyNGXEgeUqKkB5ccK83',
 '63wjoROpeh5f11Qm93UiJ1',
 '0TImkz4nPqjegtVSMZnMRq',
 '4eAOcbAXIF4BmbN6E1QIlw',
 '1l7ZsJRRS8wlW3WfJfPfNS']

In [54]:
df_artists[df_artists['artist_id'].isin(artists_ids)]

,artist_pid,artist_name,artist_id
7,7,Destiny's Child,1Y8cdNmUJH7yBTd9yOvr5i
179,179,TLC,0TImkz4nPqjegtVSMZnMRq
662,662,Ashanti,5rkVyNGXEgeUqKkB5ccK83
671,671,Christina Aguilera,1l7ZsJRRS8wlW3WfJfPfNS
1059,1059,Leona Lewis,5lKZWd6HiSCLfnDGrq9RAm
1875,1875,Whitney Houston,6XpaIBNiVzIetEPCWDvAFP
3158,3158,Brandy,05oH07COxkXKIMt6mIPRee
4586,4586,Christina Milian,4eAOcbAXIF4BmbN6E1QIlw
4678,4678,Keri Hilson,63wjoROpeh5f11Qm93UiJ1


<br>

Observando los resultamos, vemos que existe relación entre *Mariah Carey* y los artistas obtenidos.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>